# Cargue de información

## Bibliotecas

In [10]:
# Descarga de las bibliotecas a utilizar

#%pip install nltk
#%pip install googletrans==4.0.0-rc1
#%pip install regex
#%pip install unidecode
#%pip install langdetect

In [2]:
# Importación de las bibliotecas a utilizar
import pandas as pd
from dateutil import parser
import gzip
import json
from pandas import json_normalize
import unicodedata

from langdetect import detect, LangDetectException

import regex
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [3]:
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Bases de datos - Rutas

In [4]:
# Se obtienen las rutas de los archivos
ruta_games = "../Data/originales/steam_games.json.gz"
ruta_reviews = "../Data/originales/user_reviews.json.gz"
ruta_items = "../Data/originales/users_items.json.gz"

## Funciones generales

In [5]:
# Se crea una función que abre los archivos json dañados y regresa una lista con los datos transformados.
def abrir_json_corrupto(ruta):
    '''A partir de la ruta de un archivo json corrupto comprimido en formato gzip, 
    abre el archivo y hace una limpieza de las líneas para acercarlas a un formato json'''
    # Abre el archivo JSON.GZIP para descomprimirlo y convertirlo en una lista.
    with gzip.open(ruta,"rt",encoding="utf-8", errors="replace") as archivo_original:
        lineas_items = archivo_original.readlines()

    # Se crea una lista donde se guardarán las líneas corregidas.
    data = []

    # Se revisa línea por línea y se reemplazan los datos que impiden que el JSON sea válido
    for linea in lineas_items:
        linea = linea.replace('"',"'")
        linea = linea.replace("True","true").replace("False","false")
        # linea = linea.replace("\': \'",'": "').replace("\', \'",'", "').replace("\': [{\'", '": [{"').replace("\': true, \'", '": true, "').replace("\': false, \'",'": false, "')
        linea = linea.replace("\':",'":').replace(", \'", ', "').replace("{\'", '{"').replace(": \'",': "').replace("\',",'",')
        # linea = linea.replace("\'}",'"}').replace("\\\'","'").replace("{\'", '{"').replace("': []",'": []')
        linea = linea.replace("\'}",'"}').replace("\\\'","'")
        linea = linea.strip('\n').strip("'")

        data.append(linea)

    # Se regresa la lista de las líneas después de la corrección realizada.
    return data

In [6]:
# Se crea una función con el propósito de verificar que los elementos sean json válidos
def es_json_valido(cadena):
    '''A partir de una cadena de texto, revisa si el elemento es json válido.'''
    try:
        json.loads(cadena)
        return True
    except ValueError:
        return False

# Se crea una función para validar la data de las listas de los elementos json transformados.
def validar_data(data):
    '''Valida la data de una lista, separando elementos json válidos de aquellos que no lo son.'''
    # Se crean las listas donde se guardaran los elementos separados.
    no_validos = []
    validos = []

    for elemento in data:
        if es_json_valido(elemento) == False:
            no_validos.append(elemento)
        else: validos.append(elemento)

    # Se imprime el número de elementos JSON no válidos
    print(f"EL total de elementos JSON no válidos de la data fue {len(no_validos)} de {len(data)} lo que equivale al {round((len(no_validos)/len(data)) * 100,2)} %")

    # Regresa la lista de datos válidos.
    return validos


In [7]:
# Se crea una función para cargar los elementos JSON corregidos a un dataframe de pandas.

def cargar_json_corregido(data):
    '''Toma una lista de elementos en formato JSON y los convierte a un dataframe de pandas'''
    #Convertir cada elemento de la lista a un diccionario
    diccionario = [json.loads(elemento) for elemento in data]

    # Combinar los diccionarios en uno solo
    data_json = json.dumps(diccionario, indent=2)
    
    # Se convierten a un dataframe de pandas.
    n_data = pd.read_json(data_json)

    return n_data


In [8]:
# Se define una función para revisar que los datos de las fechas sean válidos:
def es_fecha(fecha):
    try:
        n_fecha = parser.parse(fecha)
        return n_fecha
    except ValueError:
        return None

In [9]:
# Se crea una función para guardar los nuevos archivos

def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  # Toma el dataframe dado y lo convierte en un archivo json.
  archivo_json = DataFrame.to_json()
  # Comprime el archivo json en formato gzip.
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

# Apertura y revisión de las bases de datos

Información necesaria para las consultas: 
- Año y horas jugadas por juego y género del juego.
- Horas jugadas por usuario, género y años en los que jugó.
- Recomendaciones por juego (recommend y comentarios)
- Año y número de comentarios positivos, negativos o neutrales.

## Games

In [15]:
# Se abre el archivo de "Steam Games" y se convierte a un dataframe de pandas.
games = pd.read_json(ruta_games,compression="gzip",lines = True)
# Se obtiene la visualización de los primeros datos de la base.
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [16]:
# Se eliminan las filas cuyos datos sean todos nulos (ya que no nos proporcionan información sobre el juego).
games.dropna(how = "all", inplace = True)

In [17]:
# Se deciden eliminar las columnas "publisher", "tags","url","reviews_url","early_access", "title","specs","price","developer" ya que no proporcionan información relevante para las consultas a realizar o resulta redundante.
games.drop(columns=["publisher", "tags","url","reviews_url","early_access","title","specs","price","developer"],inplace=True)

In [19]:
# Se eliminan los valores vacíos de "release_date", ya que no nos proporcionan información relevante sobre el juego.
games = games.dropna(subset=["release_date"])

In [21]:
# Se revisa que los datos restantes para la columna "release_date" sean fechas válidas dejando la información en una nueva columna llamada "review_date"
games.insert(3,"review_date",games["release_date"].apply(es_fecha))

In [23]:
# Se elimina la columna de "release_date", se cambia el nombre de la nueva columna ("review_date") a "release_date" y se eliminan los datos nulos. 
games.drop(columns="release_date",inplace=True)
games.rename(columns={"review_date":"release_date"},inplace=True)
games.dropna(subset=["release_date"],inplace=True)

In [26]:
# Se eliminan los valores nulos de id.
games.dropna(subset=["id"],inplace=True)

In [28]:
# Se transforman los valores de id a entero
games["id"] = games["id"].astype(int)

In [30]:
# Se deciden eliminar todos los valores nulos de "genres"
games.dropna(subset=["genres"],inplace=True)

In [32]:
# Se aseguran que los datos estén en los formatos adecuados
games["release_date"] = pd.to_datetime(games["release_date"])
games["id"] = games["id"].astype(int)

In [33]:
# Se revisa la información obtenida.
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28660 entries, 88310 to 120443
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28660 non-null  object        
 1   app_name      28659 non-null  object        
 2   release_date  28660 non-null  datetime64[ns]
 3   id            28660 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 1007.6+ KB


In [31]:
# Se observa el dataframe obtenido.
games.head()

,genres,app_name,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540


In [35]:
# Se guarda la base de datos corregida
guardar_archivo(games,"../Data/corregida/r_games.json.gzip")

In [37]:
# Se observa que el dataset cargue correctamente.
games_n = pd.read_json("../Data/corregida/r_games.json.gzip",compression="gzip",convert_dates=['release_date'],date_unit="ms")
games_n.head()

,genres,app_name,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540


## Reviews

## Items